In [17]:
#Section1 Load Dataset
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [18]:
merged_dataset = pd.read_csv("C:/Users/Muhammad Mitkar/Desktop/LSN/merged_datasetokis.csv")

In [19]:
print(merged_dataset.info())
print(merged_dataset.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32337 entries, 0 to 32336
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      32337 non-null  object
 1   Language  32337 non-null  object
dtypes: object(2)
memory usage: 505.4+ KB
None
                                                Text  Language
0  klement gottwaldi surnukeha palsameeriti ning ...  Estonian
1  sebes joseph pereira thomas  på eng the jesuit...   Swedish
2  ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...      Thai
3  விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...     Tamil
4  de spons behoort tot het geslacht haliclona en...     Dutch


In [20]:
print(merged_dataset.describe(include='all'))

                                                     Text Language
count                                               32337    32337
unique                                              32126       30
top     haec commentatio automatice praeparata res ast...  English
freq                                                   48     2385


# EDA AND Text Processing on Dataset

In [22]:
merged_dataset.dropna(inplace=True)
merged_dataset.drop_duplicates(inplace=True)

In [23]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text
# Apply text cleaning to the 'Text' column
merged_dataset['Text'] = merged_dataset['Text'].apply(clean_text)


In [24]:
from sklearn.preprocessing import LabelEncoder

# Encode the language labels
label_encoder = LabelEncoder()
merged_dataset['Language_Encoded'] = label_encoder.fit_transform(merged_dataset['Language'])


# Generating NLP based Features

In [26]:
# Split the dataset into features and labels
texts = merged_dataset['Text'].values
labels = merged_dataset['Language_Encoded'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [27]:
# Initialize and fit the Tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
# Pad sequences to ensure uniform length
X_train_padded = pad_sequences(X_train_seq, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=100, padding='post', truncating='post')


# Defining and Training MLP Model

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Define the MLP model
mlp_model = Sequential([
    Flatten(input_shape=(100,)),  # Flatten the input
    Dense(128, activation='relu'),  # First hidden layer
    Dropout(0.3),  # Dropout for regularization
    Dense(64, activation='relu'),  # Second hidden layer
    Dropout(0.3),  # Dropout for regularization
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer (multi-class classification)
])

# Compile the model
mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


C:\Users\Muhammad Mitkar\Desktop\LSN\NEW-VENV\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
# Train the model
history = mlp_model.fit(
    X_train_padded, y_train,
    epochs=10,  # You can adjust the number of epochs
    validation_data=(X_test_padded, y_test),
    batch_size=32  # Adjust based on your system's memory capacity
)


Epoch 1/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.0558 - loss: 148.2003 - val_accuracy: 0.0735 - val_loss: 3.3349
Epoch 2/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0754 - loss: 3.6240 - val_accuracy: 0.0735 - val_loss: 3.3101
Epoch 3/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0760 - loss: 3.4040 - val_accuracy: 0.0735 - val_loss: 3.3036
Epoch 4/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0713 - loss: 3.3521 - val_accuracy: 0.0735 - val_loss: 3.3018
Epoch 5/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.0750 - loss: 3.3634 - val_accuracy: 0.0735 - val_loss: 3.3014
Epoch 6/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0732 - loss: 3.3185 - val_accuracy: 0.0735 - val_loss: 3.3011
Epoch 7/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0746 - loss: 3.3138 - val_accuracy: 0.0735 - val_loss: 3.3010
Epoch 8/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0755 - loss: 3.3099 - val_accuracy: 

# Evaluating Model Performance

In [32]:
# Evaluate the model on test data
loss, accuracy = mlp_model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0693 - loss: 3.3022
Test Loss: 3.300804615020752
Test Accuracy: 0.07345160096883774


# Training Bidirectional LSTM MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the Bidirectional LSTM model
model = Sequential([
    # Embedding Layer
    Embedding(input_dim=5000,  # Vocabulary size (must match tokenizer num_words)
              output_dim=128,  # Embedding vector size
              input_length=100),  # Max sequence length (must match padding)
    
    # Bidirectional LSTM Layer
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),  # Dropout for regularization
    
    # Additional LSTM layer for deeper learning
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    
    # Fully connected layer
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    # Output Layer
    Dense(len(label_encoder.classes_), activation='softmax')  # Number of classes in the dataset
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]
)


C:\Users\Muhammad Mitkar\Desktop\LSN\NEW-VENV\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 220s 264ms/step - accuracy: 0.3315 - loss: 2.2738 - val_accuracy: 0.8469 - val_loss: 0.5330
Epoch 2/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 163s 203ms/step - accuracy: 0.8272 - loss: 0.6096 - val_accuracy: 0.9034 - val_loss: 0.3455
Epoch 3/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 191s 238ms/step - accuracy: 0.8894 - loss: 0.3874 - val_accuracy: 0.8824 - val_loss: 0.3716
Epoch 4/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 166s 207ms/step - accuracy: 0.9093 - loss: 0.3196 - val_accuracy: 0.9195 - val_loss: 0.2760
Epoch 5/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 152s 190ms/step - accuracy: 0.9175 - loss: 0.2831 - val_accuracy: 0.8623 - val_loss: 0.3793
Epoch 6/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 173s 214ms/step - accuracy: 0.9143 - loss: 0.2799 - val_accuracy: 0.9186 - val_loss: 0.2896
Epoch 7/10
804/804 ━━━━━━━━━━━━━━━━━━━━ 183s 227ms/step - accuracy: 0.9221 - loss: 0.2599 - val_accuracy: 0.9185 - val_loss: 0.2998
Epoch 8/10
294/804 ━━━━━━━━━━━━━━━━━━━━ 1:45 207ms/step - accuracy: 0.9233 -

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predict and evaluate classification performance
y_pred = np.argmax(model.predict(X_test_padded), axis=1)

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


# Testing Input on LSTM MODEL

In [ ]:
def predict_language(input_text, tokenizer, model, label_encoder, max_len=100):
    """
    Predict the language of a given input text.

    Args:
        input_text (str): The text to classify.
        tokenizer (Tokenizer): The fitted tokenizer used for text preprocessing.
        model (Sequential): The trained model.
        label_encoder (LabelEncoder): The fitted label encoder for decoding labels.
        max_len (int): Maximum sequence length for padding (default is 100).

    Returns:
        str: Predicted language.
    """
    # Preprocess the input text
    def clean_text(text):
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\d+', '', text)      # Remove numbers
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        return text

    cleaned_text = clean_text(input_text)
    text_sequence = tokenizer.texts_to_sequences([cleaned_text])
    text_padded = pad_sequences(text_sequence, maxlen=max_len, padding='post', truncating='post')

    # Predict the language
    prediction = model.predict(text_padded)
    predicted_label = np.argmax(prediction, axis=1)[0]
    predicted_language = label_encoder.inverse_transform([predicted_label])[0]

    return predicted_language

# Example usage
input_text = " 彼はずっと世界中を旅してさまざまな文化を体験."  # Replace with your input text
predicted_language = predict_language(input_text, tokenizer, model, label_encoder)
print(f"The predicted language is: {predicted_language}")


# Trainiing GRU MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Define the GRU model
gru_model = Sequential([
    # Embedding Layer
    Embedding(input_dim=5000,  # Vocabulary size (must match tokenizer num_words)
              output_dim=128,  # Embedding vector size
              input_length=100),  # Max sequence length (must match padding)
    
    # Bidirectional GRU Layer
    Bidirectional(GRU(128, return_sequences=True)),  # GRU with return_sequences for stacking
    Dropout(0.3),  # Dropout for regularization
    
    # Additional GRU layer
    Bidirectional(GRU(64)),
    Dropout(0.3),
    
    # Fully connected layer
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    # Output Layer
    Dense(len(label_encoder.classes_), activation='softmax')  # Number of classes
])

# Compile the GRU model
gru_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the GRU model summary
gru_model.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the GRU model
gru_history = gru_model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]
)

# Evaluate the GRU model on the test set
test_loss, test_accuracy = gru_model.evaluate(X_test_padded, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predict and evaluate classification performance
y_pred = np.argmax(gru_model.predict(X_test_padded), axis=1)

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


# Testing Prediction with help of GRU  model

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Sample input text (change this to test different languages)
sample_text = [ "HELLO MY NAME IS MUHAMMAD "]  # Exampช่วยพูดช้าๆได้ไหมle in French

# Step 2: Tokenize the input text
sample_text_sequences = tokenizer.texts_to_sequences(sample_text)

# Step 3: Pad the sequences to match the input length of the model
sample_text_padded = pad_sequences(sample_text_sequences, maxlen=100, padding='post', truncating='post')

# Step 4: Use the GRU model to predict
prediction = gru_model.predict(sample_text_padded)

# Step 5: Decode the predicted label
predicted_label_index = np.argmax(prediction, axis=1)[0]
predicted_language = label_encoder.inverse_transform([predicted_label_index])[0]

print(f"Input text: {sample_text[0]}")
print(f"Predicted language: {predicted_language}")


# Saving the Model

In [ ]:
# Save the model to an HDF5 file
gru_model.save('gru_language_detection_model.h5')


In [ ]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json

# Save the tokenizer to a JSON file
tokenizer_json = tokenizer.to_json()
with open("tokenizer.json", "w") as f:
    json.dump(tokenizer_json, f)

print("Tokenizer saved to 'tokenizer.json'.")


In [ ]:
import pickle  # Import the pickle module

# Save the label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("Label encoder saved.")
